# TO DO
Change parallel arrays for checking accuracy to data frame instead <br>

BEFORE PUTTING ON GITHUB!!! <br>
Very small step in prediction mesh <br>
Range of k-values to try <br>
File name (not the partial data file) <br>

In [ ]:
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.gridspec as gridspec
from sklearn import neighbors
import pandas as pd
from pandas import DataFrame, Series
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import operator

# Format data
Indicate the name of the csv file with a column to each parameter, a row to each trial, and both row and column headers. The classification indicator must be the last column and contain either the integer 0 or 1 only.

In [ ]:
inputDataFileName = 'ParameterData.csv'

In [ ]:
# Load data into data frame, ignoring the 1st column of trial indices
with open(inputDataFileName) as f:
    ncols = len(f.readline().split(','))

parameterDF = pd.read_csv(inputDataFileName, delimiter=',', index_col=0, header=0 )

In [ ]:
# Delete the indexing column and reset the indices
parameterDF = parameterDF.reset_index()
del parameterDF['index']

In [ ]:
# Normalize the data
params = parameterDF.columns.tolist() # List of labels for indexing
normDF = DataFrame()

for param in params[:-1]: # Normalize each parameter column (not classification column)
    normalizedData = ( parameterDF[param] - parameterDF[param].mean() ) / parameterDF[param].std()
    normDF = pd.concat( [normDF, Series(normalizedData) ], axis=1 )
    normDF.rename( columns={ 0:param } )

# Add the classification column back in
normDF = pd.concat( [normDF, parameterDF[ params[-1] ] ], axis=1 )

In [ ]:
# Group by classification
groups = normDF.groupby( normDF.columns[-1] )
successGroup = groups.get_group(1)
failureGroup = groups.get_group(0)

# Find optimal k and weight method for each pair of parameters

In [ ]:
numParams = len(params)-1 # For looping through parameters

# Set the range of values of k around the typical standard for k=sqrt(n)
# And only look at the odd values for k
kMid = (int)( sqrt(parameterDF.shape[0]) ) # k nearest neighbors

if kMid%2 is 0: # Want odd k
    kMid = kMid + 1
    
kBeg = kMid - 50
kEnd = kMid + 50
ks = np.arange(kBeg, kEnd, 2)

In [ ]:
bestClassifiersByWeight = []
overallBestClassifiers = []
overallBestKs = []
overallBestAccValues = []
overallBestWeights = []

accScoresByWeight = []
allAccScores = [] 

xMins = []
xMaxs = []
yMins = []
yMaxs = []

comparisons = []

listOfWeights = ['uniform', 'distance']

# Create list of comparison labels
for xNum in range(numParams):
    for yNum in range(numParams):
        if xNum < yNum:
            comparisons.append( params[xNum]+' vs '+params[yNum] )
            
# Fit and test the classifiers   
for weight in listOfWeights:
    
    # Now inside the loop for each weight
    print(str(weight)+':')
    maxAccValues = []
    bestKs =[]
    accScoresByParams = []
    bestClassifiers = []
    
    for rowNum in range(numParams):    
    
        for colNum in range(numParams): 
        
            # Now inside the loop for each set of params
        
            if rowNum < colNum: # Exclude repeated plots 
            
                # Now inside the loop for each set of params we use (without the ones we don't use)
                print()
                print( '     '+str(params[colNum]) + ' vs ' + str(params[rowNum]) + ':')                
            
                XTrains = []
                XTests = []
                yTrains = []
                yTests = []

                accuracyScores = [] 
                classifiers = []

                # Get data
                x1 = normDF[ params[colNum] ] # x-axis
                x2 = normDF[ params[rowNum] ] # y-axis
                X = np.column_stack( (x1, x2) )

                # Get overall min and max values for plotting and prediction testing
                xMin, xMax = x1.min(), x1.max() 
                yMin, yMax = x2.min(), x2.max()            
                xMins.append(xMin), xMaxs.append(xMax), yMins.append(yMin), yMaxs.append(yMax)           

                # Split data into training and testing samples
                XTrain, XTest, yTrain, yTest = train_test_split(X, normDF[ params[-1] ])            
                XTrains.append(XTrain), XTests.append(XTest), yTrains.append(yTrain), yTests.append(yTest)            
                # USING SAME DATA FOR TRAINING VS TESTING FOR EACH K THAT WE TRY

                # loop for each k
                for k in ks:

                    # Now inside the loop for each k for the set of params
                    
                    # Fit the classifier
                    classifier = neighbors.KNeighborsClassifier( k, weights=weight ).fit(XTrain, yTrain)
                    classifiers.append(classifier)                

                    # Predict test data and find accuracy for this k
                    predicted = classifier.predict( XTest )
                    acc = accuracy_score( yTest, predicted )
                    accuracyScores.append( acc )  
                    print('          k = '+str(k)+'  accuracy = '+str(acc))

                # Now back to the loop for each set of params that we use
            
                # Find the best accuracy for this set of params, get the cooresponding k and classifier
                maxAccIndex, maxAccValue = max(enumerate(accuracyScores), key=operator.itemgetter(1))
                bestK = ks[maxAccIndex]
                maxAccValues.append( maxAccValue ) # actual accuracy
                bestKs.append( bestK ) # just to know 
                bestClassifiers.append( classifiers[maxAccIndex] ) # for plotting  
                accScoresByParams.append(accuracyScores)
                
                print()
                print( '          Max accuracy at k=' + str(bestK) + ' at ' + str(maxAccValue) )
                
            # Back to the loop for each set of params (with repeats)
        
        
    # Back to the loop for each weight 
    accScoresByWeight.append(accScoresByParams)
    overallBestKs.append(bestKs)
    bestClassifiersByWeight.append(bestClassifiers)
    overallBestAccValues.append(maxAccValues)


In [ ]:
if max(overallBestAccValues[0]) > max(overallBestAccValues[1]):
    overallBestAccIndex, overallBestAccValue = max(enumerate(overallBestAccValues[0]), key=operator.itemgetter(1))
    overallBestK = overallBestKs[0][overallBestAccIndex]
    overallBestWeight = listOfWeights[0] 
    
else:
    overallBestAccIndex, overallBestAccValue = max(enumerate(overallBestAccValues[1]), key=operator.itemgetter(1))
    overallBestK = overallBestKs[1][overallBestAccIndex]
    overallBestWeight = listOfWeights[1]

print('Maximum accuracy achieved with:')
print('     '+str( np.hstack( (comparisons, comparisons))[overallBestAccIndex]))
print('     accuracy = '+str(overallBestAccValue))
print('     k = '+str(overallBestK))
print('     weight = '+str(overallBestWeight))

# Plot training data & prediction boundaries with optimized accuracy

In [ ]:
overallBestClassifiers = []
index = 0
bestClassifiers = [ (a, b) for (a, b) in np.column_stack( (overallBestAccValues[0], overallBestAccValues[1]) ) ] 
for (a, b) in bestClassifiers:
    if a > b:
        overallBestClassifiers.append( bestClassifiersByWeight[0][index] )
    else:
        overallBestClassifiers.append( bestClassifiersByWeight[1][index] )
    index += 1

In [ ]:
h = 0.005  # Step size in the prediction mesh
colMap = ListedColormap( ['#F08080', '#98FB98'] ) # Prediction mesh colors
meshXs = []
meshYs = []
meshPredictions = [] 

# Predict a meshgrid of data points
for num in range( len(overallBestClassifiers) ):
    meshX, meshY = np.meshgrid(np.arange(xMins[num]-1, xMaxs[num]+1, h),
                               np.arange(yMins[num]-1, yMaxs[num]+1, h))
    meshXs.append(meshX), meshYs.append(meshY)
    meshPredicted = overallBestClassifiers[num].predict(np.c_[meshX.ravel(), meshY.ravel()])
    meshPredicted = meshPredicted.reshape(meshX.shape)
    meshPredictions.append(meshPredicted)

In [ ]:
# Prepare the figure for plotting
plt.rcParams.update({'figure.autolayout': False})
plt.figure( figsize=(15,15) )
G = gridspec.GridSpec(numParams, numParams)
plt.subplots_adjust( hspace=0.1, wspace=0.1 ) 
plt.rc('font', size=20)
plt.suptitle('Optimized KNN Classifiers', x=0.5, y=0.9)
axarr = []
indices = [] # For indexing row and column labels

for xNum in range(numParams):
    for yNum in range(numParams):
        if xNum < yNum:
            indices.append( (params[xNum],params[yNum]) ) 
            ax = plt.subplot( G[yNum-1, xNum] )
            axarr.append(ax)
            if xNum is 0:
                plt.ylabel( params[yNum] )
            if xNum is not 0:
                plt.yticks( visible=False )
            if yNum is numParams-1:
                plt.xlabel( params[xNum] )
                plt.xticks( rotation=45 )
            if yNum is not numParams-1:
                plt.xticks( visible=False ) 
                         
# Plot 
for num in range(len(indices)):                
    # Plot decision boundary
    axarr[num].pcolormesh(meshXs[num], meshYs[num], meshPredictions[num], cmap=colMap)
    
    # Plot the training data 
    xSuccesses = successGroup[ indices[num][1] ].head(200)
    ySuccesses = successGroup[ indices[num][0] ].head(200)
    axarr[num].scatter( xSuccesses, ySuccesses, marker='o', color='#006400', alpha=0.7, label='success' )
    
    xFailures = failureGroup[ indices[num][1] ].head(200)
    yFailures = failureGroup[ indices[num][0] ].head(200)
    axarr[num].scatter( xFailures, yFailures, marker='s', color='#FF0000', alpha=0.7, label='failure')
            
    axarr[num].set_xlim( -3,3 )
    axarr[num].set_ylim( -3,3 ) 
            
#plt.legend(bbox_to_anchor=(1,1), loc=0, borderaxespad=-4)
#plt.legend(bbox_to_anchor=(1,1), loc=1, borderaxespad=-4)
plt.legend(bbox_to_anchor=(1,1), loc=2, borderaxespad=-11)
plt.savefig('KNNPlotOptimized.png')

# Graph prediction accuracy

In [ ]:
pos = list( range( len(comparisons) ) )
width = 0.4
fig, ax = plt.subplots( figsize=(13, 8) )
plt.rcParams.update({'figure.autolayout': False})
plt.rc('font', size=16)
plt.barh( pos, overallBestAccValues[0], width, color='#EEFF24', alpha=0.7, label=listOfWeights[0] )
plt.barh([p+width for p in pos], overallBestAccValues[1], width, color='#F78F1E', label=listOfWeights[1])
ax.set_xlabel('Accuracy')
ax.set_title('Accuracy vs Weight Method')
ax.set_xticks(np.arange(0, 1.1, .1))
ax.set_yticks([p+width for p in pos])
ax.set_yticklabels(comparisons)
plt.ylim( min(pos)-width, max(pos)+width*3)
plt.xlim( 0, 1.0 )

# Find pos to print the text at the end of each pair of bars by finding max of each pair of bars
overallBestAccValuesReshaped = np.column_stack( (overallBestAccValues[0], overallBestAccValues[1]) )
maxOverallBestAccValues = [ max(coord) for coord in overallBestAccValuesReshaped ]
for yPos, xPos in enumerate(overallBestAccValues[0]):
    ax.text(xPos+0.01, yPos+0.01, str(xPos))
    
for yPos, xPos in enumerate(overallBestAccValues[1]):
    ax.text(xPos+0.01, yPos+0.5, str(xPos))
    
plt.grid()
handles, labels = ax.get_legend_handles_labels()
lgd = plt.legend(handles, labels, bbox_to_anchor=(1.05, 0.55), loc=2, borderaxespad=0.)
plt.savefig('AccuracyVsWeight_plot1.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

In [ ]:
pos = list( range( len(comparisons) ) )
width = 0.4
fig, ax = plt.subplots( figsize=(13, 8) )
plt.rcParams.update({'figure.autolayout': True})
plt.rc('font', size=16)
plt.barh( pos, overallBestAccValues[0], width, color='#EEFF24', alpha=0.7, label=listOfWeights[0] )
plt.barh([p+width for p in pos], overallBestAccValues[1], width, color='#F78F1E', label=listOfWeights[1])
ax.set_xlabel('Accuracy')
ax.set_xticks(np.arange(0, 1.1, .1))
ax.set_yticks([p+width for p in pos])
ax.set_yticklabels(comparisons)
plt.ylim( min(pos)-width, max(pos)+width*3)
plt.xlim( 0, 1.0 )

# Find pos to print the text at the end of each pair of bars by finding max of each pair of bars
overallBestAccValuesReshaped = np.column_stack( (overallBestAccValues[0], overallBestAccValues[1]) )
maxOverallBestAccValues = [ max(coord) for coord in overallBestAccValuesReshaped ]

for yPos, xPos in enumerate(maxOverallBestAccValues):
    ax.text(xPos+0.01, yPos+0.25, str(xPos))    
    
plt.grid()
diffInBestAccValues = [ (a-b) for (a,b) in overallBestAccValuesReshaped ]
ax.set_title('Accuracy vs Weight Method (max diff='+str(max(diffInBestAccValues))+')')
handles, labels = ax.get_legend_handles_labels()
lgd = plt.legend(handles, labels, bbox_to_anchor=(1.05, 0.55), loc=2, borderaxespad=0.)
plt.savefig('AccuracyVsWeight_plot2.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

In [ ]:
# Graph the accuracy (horizontal bar chart)
plt.rcParams.update({'figure.autolayout': True})
yAxis = np.arange(len(comparisons))
plt.rc('font', size=18)
plt.figure( figsize=(13,8) )
plt.barh(yAxis, overallBestAccValues[0], align='center')

for yPos, xPos in enumerate(overallBestAccValues[0]):
    plt.gca().text(xPos+0.01, yPos-0.2, str(xPos))
    
plt.yticks(yAxis, comparisons)
plt.grid()
plt.xlim(0, 1.0)
plt.xlabel('Accuracy')
plt.title('Uniform Weight Method - Best Accuracy Achieved')
plt.savefig('UniformAccuracy.png') 

In [ ]:
# Graph the accuracy (horizontal bar chart)
plt.rcParams.update({'figure.autolayout': True})
yAxis = np.arange(len(comparisons))
plt.rc('font', size=18)
plt.figure( figsize=(13,8) )
plt.barh(yAxis, overallBestAccValues[1], align='center')

for yPos, xPos in enumerate(overallBestAccValues[1]):
    plt.gca().text(xPos+0.01, yPos-0.2, str(xPos))

plt.yticks(yAxis, comparisons)
plt.grid()
plt.xlim(0, 1.0)
plt.xlabel('Accuracy')
plt.title('Distance Weight Method - Best Accuracy Achieved')
plt.savefig('DistanceAccuracy.png') 

In [ ]:
comparisonsWOspaces = comparisons
for comparison in comparisonsWOspaces:
    comparison=comparison.replace(' ', '')
    comparison=comparison.replace('vs', 'Vs')

In [ ]:
for index in range( len(comparisons) ):

    plt.figure( figsize=(10, 5) )
    plt.title(comparisons[index])
    plt.plot( ks, accScoresByWeight[0][index], linewidth=3, label=listOfWeights[0] )
    plt.plot( ks, accScoresByWeight[1][index], linewidth=3, label=listOfWeights[1] )
    plt.xlabel('K')
    plt.ylabel('Accuracy')
    plt.xlim( min(ks), max(ks) )
    handles, labels = plt.gca().get_legend_handles_labels()
    lgd = plt.legend(handles, labels, bbox_to_anchor=(1.05, .55), loc=2, borderaxespad=0.)
    plt.savefig('KvsAccuracy_'+comparisonsWOspaces[index]+'.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()

In [ ]:
plt.figure( figsize=(8, 7) )
plt.title('K vs Accuracy ('+listOfWeights[0]+' weight)')

for index in range( len(comparisons) ):
    plt.plot( ks, accScoresByWeight[0][index], linewidth=3, label=comparisons[index] )
    
plt.xlim( min(ks), max(ks) )
plt.ylim( min( min(accScoresByWeight[0]) )-0.01, max( max(accScoresByWeight[0]) )+0.01 )
handles, labels = plt.gca().get_legend_handles_labels()
lgd = plt.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig('KvsAccuracy_'+listOfWeights[0]+'.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

In [ ]:
plt.figure( figsize=(8, 7) )
plt.title('K vs Accuracy ('+listOfWeights[1]+' weight)')

for index in range( len(comparisons) ):
    plt.plot( ks, accScoresByWeight[1][index], linewidth=3, label=comparisons[index] )
    
plt.xlim( min(ks), max(ks) )
plt.ylim( min( min(accScoresByWeight[1]) )-0.01, max( max(accScoresByWeight[1]) )+0.01 )
handles, labels = plt.gca().get_legend_handles_labels()
lgd = plt.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig('KvsAccuracy_'+listOfWeights[1]+'.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

In [ ]:
# Prepare the figure for plotting
plt.rcParams.update({'figure.autolayout': False})
plt.figure( figsize=(15,15) )
plt.suptitle('K vs Accuracy', x=0.5, y=0.9)
G = gridspec.GridSpec(numParams, numParams)
plt.subplots_adjust( hspace=0.2, wspace=0.8 ) 
plt.rc('font', size=18)
axarr = []
indices = [] # For indexing row and column labels

for xNum in range(numParams):
       
    for yNum in range(numParams):
        
        if xNum < yNum:            
            indices.append( (params[xNum],params[yNum]) ) 
            ax = plt.subplot( G[yNum-1, xNum] )
            axarr.append(ax)
            
            if xNum is yNum-1:
                ax.yaxis.set_label_coords(1.8, 0.5)
                plt.ylabel( params[yNum], rotation=0 )
                
            #if xNum is not 0:
            #    plt.yticks( visible=False )
                
            if yNum is numParams-1:
                plt.xlabel( params[xNum] )
                plt.xticks( rotation=45 )
                
            if yNum is not numParams-1:
                plt.xticks( visible=False )        
               

# Plot 
for index in range( len(indices) ):    
    
    axarr[index].plot( ks, accScoresByWeight[0][index], color='#006400', linewidth=3, alpha=0.7, label=listOfWeights[0] )
    axarr[index].plot( ks, accScoresByWeight[1][index], color='#FF0000', linewidth=3, alpha=0.7, label=listOfWeights[1] )

handles, labels = plt.gca().get_legend_handles_labels()
lgd = plt.legend(handles, labels, bbox_to_anchor=(1.7,1.7), loc=2, borderaxespad=-8)
plt.savefig('KvsAccuracy_gridspec.png', bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()    

In [ ]:
# Prepare the figure for plotting
plt.rcParams.update({'figure.autolayout': True})

size = int(len(comparisons)/4)+1
f, axarr = plt.subplots( 4, size, figsize=(20,20), sharex=True, sharey=False)
plt.subplots_adjust( hspace=1 ) 
plt.rc('font', size=20)
plt.suptitle('K vs Accuracy', x=0.5, y=1.02, size=26)
             
# Plot
for index in range( len(comparisons) ):  

    if index < size:
        row = 0
    elif index < size*2:
        row = 1
    elif index < size*3:
        row = 2
    else:
        row = 3
        
    axarr[row][index-row*size].plot( ks, accScoresByWeight[0][index], linewidth=3, color='#EEFF24', alpha=0.7 )
    axarr[row][index-row*size].plot( ks, accScoresByWeight[1][index], linewidth=3, color='#F78F1E' )
    axarr[row][index-row*size].set_xlabel( 'K' )
    axarr[row][index-row*size].set_ylabel( 'Accuracy' )
    axarr[row][index-row*size].set_xlim( min(ks), max(ks) )
    axarr[row][index-row*size].set_title( str(comparisons[index]) )
    
    for tick in axarr[row][index-row*size].get_xticklabels():
        tick.set_rotation(45)

plt.xticks( rotation=45 )
plt.savefig('KvsAccuracy_subplots.png')                
plt.show()